In [37]:
import pandas as pd
import pandas_ta as ta
from datetime import datetime, time

headers = ['Date','Time','Open','High','Low','Close','Volume']
df = pd.read_csv("QQQ4h.txt", header = None)
df.columns=headers
#
period = 30  #length of moving average used to compute stdevs   ..was 30
#
df['OHLC'] = (df.Open+df.High+df.Low+df.Close)/4
df['ma_30'] = df.Close.rolling(period).mean()
df['std'] = df.Close.rolling(period).std()
df['upper_bband'] = df.ma_30 + (2*df['std'])
df['lower_bband'] = df.ma_30 - (2*df['std'])
df['rsi'] = ta.rsi(df.Close, length = 18)
df['atr'] = round(ta.atr(high = df.High, low = df.Low, close = df.Close, length = 10), 2)
dfLen = len(df)
df

,Date,Time,Open,High,Low,Close,Volume,OHLC,ma_30,std,upper_bband,lower_bband,rsi,atr
0,06/22/2018,08:00,171.70,171.76,170.1200,170.8300,12473457,171.102500,NaN,NaN,NaN,NaN,NaN,NaN
1,06/22/2018,12:00,170.84,171.32,170.4300,170.5600,13524735,170.787500,NaN,NaN,NaN,NaN,NaN,NaN
2,06/25/2018,08:00,169.23,169.44,166.5100,166.7100,26678973,167.972500,NaN,NaN,NaN,NaN,NaN,NaN
3,06/25/2018,12:00,166.70,167.19,165.3700,166.8800,36711009,166.535000,NaN,NaN,NaN,NaN,NaN,NaN
4,06/26/2018,08:00,167.49,168.13,166.8900,167.5500,12953007,167.515000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,06/22/2022,12:00,281.32,284.54,280.2300,280.6600,22349764,281.687500,291.834667,14.988986,321.812639,261.856694,42.302901,6.15
2014,06/23/2022,08:00,283.20,285.14,280.3100,282.9100,20720585,282.890000,291.117333,14.879970,320.877274,261.357393,44.642245,6.02
2015,06/23/2022,12:00,282.93,285.72,280.4100,284.9000,22371857,283.490000,290.434333,14.671040,319.776414,261.092253,46.667260,5.95
2016,06/24/2022,08:00,287.77,293.41,287.5001,291.7466,21352159,290.106675,289.862220,14.254456,318.371131,261.353309,52.938596,6.20


In [38]:
signal = [0]*dfLen
for i in range(0, dfLen):
    if df.Close[i]<df.lower_bband[i] and df.rsi[i]<30:#close below band, oversold
        signal[i] = 1
    elif df.Close[i]>df.upper_bband[i] and df.rsi[i]>70:
        signal[i] = 2
df['signal'] = signal

In [39]:
df['signal'].value_counts()
#df
#df[df['signal'] == 1].index.values

0    1968
2      34
1      16
Name: signal, dtype: int64

In [46]:
import plotly.graph_objects as go
def plotSignal(df):
    fig = go.Figure(data=[go.Candlestick(x=df.index, open=df['Open'],  high=df['High'], low=df['Low'],
                                         close=df['Close'], name='QQQ4h')])
    fig.add_trace(go.Line(x=df.index, y=df.ma_30, name='SMA30'))
    fig.add_trace(go.Line(x=df.index, y=(df.upper_bband), name='upper bband'))
    fig.add_trace(go.Line(x=df.index, y=(df.lower_bband), name='lower bband'))
    fig.show()
    
df1 = df[520:1520]
plotSignal(df1)

In [41]:
def SIGNAL():
    return df.signal

In [51]:
from backtesting import Strategy, Backtest
import numpy as np

class myStrat(Strategy):
    def init(self):
        super().init()
        self.signal1=self.I(SIGNAL)     
        
    def next(self):
        super().next()
        trailSL = self.data.Close[-1]*.0045
        #slatr = 1.5*self.data.ATR[-1]
        for trade in self.trades:
            if trade.is_long:
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - trailSL)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + trailSL)
        
        if self.signal1==1:# and len(self.trades)==0: #Long Signal
            self.buy()
            
        if self.signal1==2:# and len(self.trades)==0: #Short Signal
            self.sell()
            
bt = Backtest(df, myStrat, cash=100_000, margin = 1/1, commission=.000)
stat = bt.run()
stat
#bt.plot(show_legend=False, superimpose = True)

C:\Users\desma\AppData\Local\Temp/ipykernel_23232/2511624366.py:25: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    2017.0
Duration                               2017.0
Exposure Time [%]                    5.500496
Equity Final [$]                131442.613805
Equity Peak [$]                 132036.646655
Return [%]                          31.442614
Buy & Hold Return [%]               72.680443
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -3.002772
Avg. Drawdown [%]                   -1.339468
Max. Drawdown Duration                  673.0
Avg. Drawdown Duration             112.470588
# Trades                                 40.0
Win Rate [%]                             57.5
Best Trade [%]                       6.914231
Worst Trade [%]                     -2.237217
Avg. Trade [%]                    